In [32]:
import pandas as pd
from bearinmind_pipeline.data_preprocessing import dataPreprocessing as dp

In [2]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [3]:
df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [40]:
file_train = pd.DataFrame(df.groupby('molecule_name')['scalar_coupling_constant'].mean())

In [43]:
file_train.reset_index(inplace=True)

In [44]:
file_train.head()

,molecule_name,scalar_coupling_constant
0,dsgdb9nsd_000001,27.170450
1,dsgdb9nsd_000002,10.755067
2,dsgdb9nsd_000003,-9.946410
3,dsgdb9nsd_000005,88.201230
4,dsgdb9nsd_000007,17.681228


In [15]:
df_smiles = pd.read_table('Smiles/all_mol.txt', header = None)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [25]:
df_smiles.columns = ['structure', 'molecule_name']

In [33]:
df_smiles = dp.parse_str_col(df_smiles, ['molecule_name'], '/', 1)

In [35]:
df_smiles = dp.parse_str_col(df_smiles, ['molecule_name'], '.', 0)

In [38]:
df_smiles.head(10)

,structure,molecule_name
0,C,dsgdb9nsd_000001
1,N,dsgdb9nsd_000002
2,O,dsgdb9nsd_000003
3,C#C,dsgdb9nsd_000004
4,C#N,dsgdb9nsd_000005
5,CC,dsgdb9nsd_000007
6,CO,dsgdb9nsd_000008
7,CC#C,dsgdb9nsd_000009
8,CC#N,dsgdb9nsd_000010
9,CC=O,dsgdb9nsd_000011


In [69]:
out_df = df_smiles.merge(file_train, how = 'left', on = 'molecule_name')

In [70]:
out_df.shape

(130775, 3)

In [71]:
out_df = out_df.dropna()

In [76]:
out_df_finale = out_df[['structure', 'scalar_coupling_constant']]

In [73]:
#out_df = out_df.reset_index()

In [77]:
out_df_finale.head(10)

,structure,scalar_coupling_constant
0,C,27.170450
1,N,10.755067
2,O,-9.946410
4,C#N,88.201230
5,CC,17.681228
6,CO,25.293834
7,CC#C,29.080592
8,CC#N,17.541745
9,CC=O,25.046757
10,NC=O,28.261469


In [82]:
out_df_finale.to_csv('gnn_train.txt', header=None, index=None, sep=' ', mode='a')